Example 4.2 of [MWL20].

[WML19] Wang, Jie, Victor Magron, and Jean-Bernard Lasserre. "TSSOS: A Moment-SOS hierarchy that exploits term sparsity." arXiv preprint arXiv:1912.08899 (2019).

In [1]:
using DynamicPolynomials
@polyvar x[1:3]
f = 1 + x[1]^4 + x[2]^4 + x[3]^4 + prod(x) + x[2]

x₁⁴ + x₂⁴ + x₃⁴ + x₁x₂x₃ + x₂ + 1

In [2]:
using SumOfSquares
SumOfSquares.Certificate.monomials_half_newton_polytope(monomials(f), tuple())

┌ Info: Precompiling SumOfSquares [4b9e565b-77fc-50a5-a571-1244f986bda1]
└ @ Base loading.jl:1260


10-element MonomialVector{true}:
 x₁²
 x₁x₂
 x₁x₃
 x₂²
 x₂x₃
 x₃²
 x₁
 x₂
 x₃
 1

In [3]:
monos = Set(monomials(x, 0:2))

Set{Monomial{true}} with 10 elements:
  1
  x₂x₃
  x₂²
  x₃
  x₃²
  x₁²
  x₁
  x₂
  x₁x₂
  x₁x₃

In [4]:
P = Set(monomials(f))
for mono in monos
    push!(P, mono^2)
end
P

Set{Monomial{true}} with 12 elements:
  x₁⁴
  x₁x₂x₃
  x₂²
  x₃⁴
  x₁²x₂²
  x₁²x₃²
  x₂
  x₃²
  x₂⁴
  x₂²x₃²
  x₁²
  1

In [5]:
const CEG = SumOfSquares.Certificate.ChordalExtensionGraph
g = CEG.LabelledGraph{monomialtype(f)}()
for mono in monos
    CEG.add_node!(g, mono)
end
for a in monos
    for b in monos
        if a != b && (a * b) in P
            CEG.add_edge!(g, a, b)
        end
    end
end
g

Monomial{true}-LabelledGraph with
Nodes:
1, x[2]*x[3], x[2]^2, x[3], x[3]^2, x[1]^2, x[1], x[2], x[1]*x[2], x[1]*x[3], 
Edges:
( x[1]*x[3], x[2] )
( x[2]*x[3], x[1] )
( x[2]^2, x[3]^2 )
( x[2]^2, x[1]^2 )
( x[2]^2, 1 )
( x[3], x[1]*x[2] )
( x[3]^2, x[2]^2 )
( x[3]^2, x[1]^2 )
( x[3]^2, 1 )
( x[1]^2, x[2]^2 )
( x[1]^2, x[3]^2 )
( x[1]^2, 1 )
( x[1], x[2]*x[3] )
( x[2], x[1]*x[3] )
( x[2], 1 )
( x[1]*x[2], x[3] )
( 1, x[2]^2 )
( 1, x[3]^2 )
( 1, x[2] )
( 1, x[1]^2 )


In [6]:
H, cliques = CEG.chordal_extension(g, CEG.GreedyFillIn())

(Monomial{true}-LabelledGraph with
Nodes:
1, x[2]*x[3], x[2]^2, x[3], x[3]^2, x[1]^2, x[1], x[2], x[1]*x[2], x[1]*x[3], 
Edges:
( x[1]*x[3], x[2] )
( x[2]*x[3], x[1] )
( x[2]^2, x[3]^2 )
( x[2]^2, x[1]^2 )
( x[2]^2, 1 )
( x[3], x[1]*x[2] )
( x[3]^2, x[2]^2 )
( x[3]^2, x[1]^2 )
( x[3]^2, 1 )
( x[1]^2, x[2]^2 )
( x[1]^2, x[3]^2 )
( x[1]^2, 1 )
( x[1], x[2]*x[3] )
( x[2], x[1]*x[3] )
( x[2], 1 )
( x[1]*x[2], x[3] )
( 1, x[2]^2 )
( 1, x[3]^2 )
( 1, x[2] )
( 1, x[1]^2 )
, Array{Monomial{true},1}[[1, x₂², x₃², x₁²], [x₂, x₁x₃], [1, x₂], [x₃, x₁x₂], [x₂x₃, x₁]])

In [7]:
P1 = Set{eltype(P)}()
for a in monos
    push!(P1, a^2)
    for bi in H.graph.neighbors[H.n2int[a]]
        b = H.int2n[bi]
        push!(P1, a * b)
    end
end
P1

Set{Monomial{true}} with 12 elements:
  x₁⁴
  x₁x₂x₃
  x₂²
  x₁²x₂²
  x₃⁴
  x₁²x₃²
  x₃²
  x₂
  x₂⁴
  x₂²x₃²
  x₁²
  1

In [8]:
P

Set{Monomial{true}} with 12 elements:
  x₁⁴
  x₁x₂x₃
  x₂²
  x₃⁴
  x₁²x₂²
  x₁²x₃²
  x₂
  x₃²
  x₂⁴
  x₂²x₃²
  x₁²
  1

In [12]:
const MP = SumOfSquares.MP
const CEG = SumOfSquares.Certificate.ChordalExtensionGraph
function monomial_sparsity_graph(monos, P)
    g = CEG.LabelledGraph{monomialtype(f)}()
    for mono in monos
        CEG.add_node!(g, mono)
    end
    for a in monos
        for b in monos
            if a != b && (a * b) in P
                CEG.add_edge!(g, a, b)
            end
        end
    end
    return g
end
function monomial_sparsity_iteration(monos, P)
    g = monomial_sparsity_graph(monos, P)
    H, cliques = CEG.chordal_extension(g, CEG.GreedyFillIn())
    P_next = Set{eltype(P)}()
    for a in monos
        push!(P_next, a^2)
        for bi in H.graph.neighbors[H.n2int[a]]
            b = H.int2n[bi]
            push!(P_next, a * b)
        end
    end
    return P_next, cliques
end
function sparsity(monos::AbstractVector{<:MP.AbstractMonomial}, sp::MonomialSparsity)
    half_monos = SumOfSquares.Certificate.monomials_half_newton_polytope(monos, tuple())
    P = Set(monos)
    for mono in half_monos
        push!(P, mono^2)
    end
    cliques = nothing
    iter = 0
    while iter < sp.k || iszero(iter)
        P_prev = P
        P, cliques = monomial_sparsity_iteration(half_monos, P_prev)
        length(P) >= length(P_prev) || error("Set of monomials should be increasing in monomial sparsity iterations.")
        length(P) == length(P_prev) && break
    end
    return P, cliques
end
function sparsity(poly::MP.AbstractPolynomial, sp::Union{SignSymmetry, MonomialSparsity})
    return sparsity(monomials(poly), sp)
end

sparsity (generic function with 3 methods)

In [16]:
sparsity(f, MonomialSparsity(1))

(Set(Monomial{true}[x₁⁴, x₁²x₂², x₁x₂x₃, x₂², x₃⁴, x₁²x₃², x₃², x₂, x₂⁴, x₂²x₃², x₁², 1]), Array{Monomial{true},1}[[x₁², x₂², x₃², 1], [x₂x₃, x₁], [x₂, 1], [x₁x₃, x₂], [x₁x₂, x₃]])

In [17]:
sparsity(f, MonomialSparsity(2))

(Set(Monomial{true}[x₁⁴, x₁²x₂², x₁x₂x₃, x₂², x₃⁴, x₁²x₃², x₃², x₂, x₂⁴, x₂²x₃², x₁², 1]), Array{Monomial{true},1}[[x₁², x₂², x₃², 1], [x₂x₃, x₁], [x₂, 1], [x₁x₃, x₂], [x₁x₂, x₃]])

In [18]:
sparsity(f, MonomialSparsity(0))

(Set(Monomial{true}[x₁⁴, x₁²x₂², x₁x₂x₃, x₂², x₃⁴, x₁²x₃², x₃², x₂, x₂⁴, x₂²x₃², x₁², 1]), Array{Monomial{true},1}[[x₁², x₂², x₃², 1], [x₂x₃, x₁], [x₂, 1], [x₁x₃, x₂], [x₁x₂, x₃]])

In [108]:
f = 1 + x[1]^2 * x[2]^4 + x[1]^4 * x[2]^2 + x[1]^4 * x[2]^4 - x[1] * x[2]^2 - 3x[1]^2 * x[2]^2

x₁⁴x₂⁴ + x₁⁴x₂² + x₁²x₂⁴ - 3x₁²x₂² - x₁x₂² + 1

In [21]:
sparsity(f, MonomialSparsity(1))

(Set(Monomial{true}[1, x₁²x₂², x₁⁴x₂², x₁x₂², x₁⁴x₂⁴, x₁²x₂⁴]), Array{Monomial{true},1}[[x₁x₂², 1], [x₁²x₂², 1], [x₁x₂], [x₁²x₂]])

In [22]:
sparsity(f, MonomialSparsity(2))

(Set(Monomial{true}[1, x₁²x₂², x₁⁴x₂², x₁x₂², x₁⁴x₂⁴, x₁²x₂⁴]), Array{Monomial{true},1}[[x₁x₂², 1], [x₁²x₂², 1], [x₁x₂], [x₁²x₂]])

In [23]:
sparsity(f, MonomialSparsity(0))

(Set(Monomial{true}[1, x₁²x₂², x₁⁴x₂², x₁x₂², x₁⁴x₂⁴, x₁²x₂⁴]), Array{Monomial{true},1}[[x₁x₂², 1], [x₁²x₂², 1], [x₁x₂], [x₁²x₂]])

In [4]:
mutable struct XORSpace{T<:Integer}
    dimension::Int
    basis::Vector{T}
    function XORSpace{T}(n) where T
        return new(0, zeros(T, n))
    end
end
function appropriate_type(n)
    if n < 8sizeof(Int)
        return Int
    elseif n < 64
        return Int64
    elseif n < 128
        return Int128
    else
        return BigInt
    end
end
function XORSpace(n)
    return XORSpace{appropriate_type(n)}(n)
end
function Base.push!(xs::XORSpace{T}, x::T) where T
    for i in eachindex(xs.basis)
        if !iszero(x & (one(T) << (i - 1)))
            if iszero(xs.basis[i])
                xs.dimension += 1
                xs.basis[i] = x
                break
            else
                x = xor(x, xs.basis[i])
            end
        end
    end
    return xs
end
function orthogonal_complement(xs::XORSpace{T}) where T
    r = Vector{T}(undef, length(xs.basis) - xs.dimension)
    k = 0
    for i in eachindex(xs.basis)
        if iszero(xs.basis[i])
            e = one(T) << (i - 1)
            x = e
            for j in 1:(i - 1)
                if !iszero(xs.basis[j] & e)
                    x |= (one(T) << (j - 1))
                end
            end
            k += 1
            r[k] = x
        end
    end
    @assert k == length(r)
    return r
end
function xor_complement(x, n, ::Type{T} = appropriate_type(n)) where T
    xs = XORSpace{T}(n)
    for xi in x
        push!(xs, xi)
    end
    return orthogonal_complement(xs)
end

xor_complement (generic function with 2 methods)

In [5]:
using Test
@test xor_complement([1], 1) == Int[]
@test xor_complement(Int[], 1) == [1]
@test xor_complement([1], 2) == [2]
@test xor_complement([2], 2) == [1]
@test xor_complement([1, 2], 2) == Int[]
@test xor_complement([1, 3], 2) == Int[]
@test xor_complement(Int[], 2) == [1, 2]
@test xor_complement([7], 3) == [3, 5]
@test xor_complement([5, 6, 3], 3) == [7]
@test xor_complement([3], 3) == [3, 4]

Test Passed

In [11]:
struct SignSymmetry <: Sparsity end
function binary_exponent(exponents, ::Type{T}) where T
    cur = zero(T)
    for exponent in exponents
        cur <<= 1
        if isodd(exponent)
            cur |= one(T)
        end
    end
    return cur
end
bin_dot(x, y) = isodd(count_ones(x & y))
function buckets_sign_symmetry(monos, r, ::Type{T}, ::Type{U}) where {T, U}
    buckets = Dict{U, Vector{eltype(monos)}}()
    for mono in monos
        exp = binary_exponent(exponents(mono), T)
        i = 1 + binary_exponent([bin_dot(ri, exp) for ri in r], U)
        if !haskey(buckets, i)
            buckets[i] = eltype(monos)[]
        end
        push!(buckets[i], mono)
    end
    return collect(values(buckets))
end
function sign_symmetry(monos::AbstractVector{<:MP.AbstractMonomial}, n, ::Type{T}) where T
    r = xor_complement((binary_exponent(exponents(mono), T) for mono in monos), n, T)
    return buckets_sign_symmetry(SumOfSquares.Certificate.monomials_half_newton_polytope(monos, tuple()),
                                 r, T, appropriate_type(length(r)))
end
function sparsity(monos::AbstractVector{<:MP.AbstractMonomial}, sp::SignSymmetry)
    n = nvariables(monos)
    return sign_symmetry(monos, n, appropriate_type(n))
end

sparsity (generic function with 2 methods)

In [118]:
sparsity(f, SignSymmetry())

r = [1, 6]


3-element Array{Array{Monomial{true},1},1}:
 [x₁, x₂]
 [x₃]
 [x₁², x₁x₂, x₂², 1]

In [119]:
@polyvar(x[1:100])

(PolyVar{true}[x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈, x₉, x₁₀  …  x₉₁, x₉₂, x₉₃, x₉₄, x₉₅, x₉₆, x₉₇, x₉₈, x₉₉, x₁₀₀],)

In [7]:
function test(n)
    @polyvar x[1:(2n)]
    sort!(sparsity(sum((x[1:2:(2n-1)] .- x[2:2:(2n)]).^2), SignSymmetry()), by = first, rev = true)
end

test (generic function with 1 method)

In [8]:
using ProfileView

┌ Info: Precompiling ProfileView [c46f51b8-102a-5cf2-8d2c-8597cb0e0da7]
└ @ Base loading.jl:1260
Gtk-Message: 15:12:45.086: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:12:45.086: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:12:51.954: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:12:51.955: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:13:13.000: Failed to load module "canberra-gtk-module"
Gtk-Message: 15:13:13.000: Failed to load module "canberra-gtk-module"


In [15]:
@profview test(100)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

In [ ]:
sparsity(sum((x[1:2:99] .- x[2:2:100]).^2), SignSymmetry())

In [115]:
f = 1 + x[1]^4 + x[1] * x[2] + x[2]^4 + x[3]^2

x₁⁴ + x₂⁴ + x₁x₂ + x₃² + 1

In [116]:
sparsity(f, SignSymmetry())

r = [1, 6]


3-element Array{Pair{Int64,Array{Monomial{true},1}},1}:
 2 => [x₁, x₂]
 3 => [x₃]
 1 => [x₁², x₁x₂, x₂², 1]